In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import StratifiedShuffleSplit, StratifiedKFold, cross_val_score
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import log_loss

In [3]:
data = pd.read_csv('train.csv')

In [4]:
v22_filled = data['v22'].fillna('*')
target = data['target']

le = LabelEncoder()
v22_filled_num = le.fit_transform(v22_filled)

ohe = OneHotEncoder()
ohe.fit(v22_filled_num.reshape(-1, 1))
sparse = ohe.transform(v22_filled_num.reshape(-1, 1))

In [5]:
# cvs = StratifiedKFold(target.as_matrix().astype(int), n_folds=5, shuffle=True, random_state=42)
cvs = StratifiedShuffleSplit(target.as_matrix().astype(int), n_iter=1, test_size=0.4, random_state=42)

estimator = ExtraTreesClassifier()

param_dist = {'max_features': ['sqrt', 'log2', 3, 5, 10, 50, 300],
              'n_estimators': [10, 20, 50, 100],
              'max_leaf_nodes': [2, 5, 10, 30, 50, 70],
              'min_samples_split': [2, 5, 7, 10, 15],
              'min_samples_leaf': [1, 2, 3, 5, 7],
              'bootstrap': [True, False],
              'criterion': ['gini', 'entropy']
             }

rs = RandomizedSearchCV(estimator, param_distributions=param_dist, n_iter=1024, scoring='log_loss', cv=cvs, n_jobs=-1,
                        random_state=9128374, refit=False)
rs.fit(sparse, target.as_matrix().astype(int))

print rs.best_score_
print rs.best_params_

-0.549221628324
{'max_leaf_nodes': 70, 'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 50, 'max_features': 50, 'criterion': 'gini', 'min_samples_split': 5}
